In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

In [7]:
file_path = file_path = "C:\\Users\\dhanv\\Downloads\\pinksheet.xlsx"
df = pd.read_excel(file_path, sheet_name="Monthly Prices", skiprows=6)

In [11]:
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

In [13]:
df['Date'] = pd.to_datetime(df['Date'].astype(str) + '-01', format='%YM%m-%d')

In [15]:
print(df.dtypes)

Date           datetime64[ns]
CRUDE_PETRO           float64
CRUDE_BRENT           float64
CRUDE_DUBAI           float64
CRUDE_WTI              object
                    ...      
NICKEL                float64
Zinc                  float64
GOLD                  float64
PLATINUM              float64
SILVER                float64
Length: 72, dtype: object


In [17]:
commodity = df.iloc[:, [0, 2, 24, 69, 71, 60, 30]].copy()
commodity.columns = commodity.columns.str.lower()

In [19]:
commodity_data = commodity.drop(columns=['date'])

In [21]:
columns_to_test = commodity_data.columns

In [23]:
non_stationary_count = 0
stationary_columns = []
non_stationary_columns = []

In [25]:
for col in columns_to_test:
    adf_result = adfuller(commodity_data[col], regression='c', autolag='AIC')
    p_value = adf_result[1]
    print(f"\nADF test result for column: {col}")
    print(f"ADF Statistic: {adf_result[0]}, p-value: {p_value}")


ADF test result for column: crude_brent
ADF Statistic: -1.507866191093539, p-value: 0.5296165197702375

ADF test result for column: soybeans
ADF Statistic: -2.4231464527418907, p-value: 0.1353097742779037

ADF test result for column: gold
ADF Statistic: 1.3430517021932997, p-value: 0.9968394353612381

ADF test result for column: silver
ADF Statistic: -1.3972947107462228, p-value: 0.5835723787985759

ADF test result for column: urea_ee_bulk
ADF Statistic: -2.510171631520914, p-value: 0.11301903181624517

ADF test result for column: maize
ADF Statistic: -2.470045106092041, p-value: 0.12293380919376795


In [29]:
if p_value > 0.05:
    non_stationary_count += 1
    non_stationary_columns.append(col)
else:
    stationary_columns.append(col)


In [34]:
print(f"\nNumber of non-stationary columns: {non_stationary_count}")
print(f"Non-stationary columns: {non_stationary_columns}")
print(f"Stationary columns: {stationary_columns}")



Number of non-stationary columns: 1
Non-stationary columns: ['maize']
Stationary columns: []


In [44]:
# Difference the data to ensure stationarity
data_diff = commodity_data.diff().dropna()

In [46]:
# Fit VAR model
model_var = VAR(data_diff)
results_var = model_var.fit(maxlags=15, ic='aic')
print(results_var.summary())

C:\Users\dhanv\anaconda3\Jupiter\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 27, Jul, 2024
Time:                     18:52:13
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                    28.6715
Nobs:                     758.000    HQIC:                   26.6206
Log likelihood:          -15509.7    FPE:                1.01462e+11
AIC:                      25.3360    Det(Omega_mle):     5.13891e+10
--------------------------------------------------------------------
Results for equation crude_brent
                      coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------
const                    0.095059         0.130624            0.728           0.467
L1.crude_brent           0.314297         0.040734            7.716           0.000
L1.soybeans              0.007564         

In [36]:
# Co-Integration Test (Johansen's Test)
# Determining the number of lags to use (you can use information criteria like AIC, BIC)
model = VAR(commodity_data)
lags = model.select_order(maxlags=10)
lag_length = lags.selected_orders['aic']  # Choosing the lag with the lowest AIC


In [40]:
# Assuming commodity_data is your dataset and it's been correctly preprocessed
johansen_test = coint_johansen(commodity_data, det_order=0, k_ar_diff=1)
print("Eigenvalues:", johansen_test.lr1)


Eigenvalues: [261.5548149  167.67790177  98.11781369  53.4617083   21.6404865
   4.01416422]


In [50]:
# You may determine the rank based on critical values or other criteria
coint_rank = sum(johansen_test.lr1 > johansen_test.cvt[:, 0])


In [54]:
# Fit VECM model
model_vecm = VECM(data_diff, k_ar_diff=1, coint_rank=coint_rank)
results_vecm = model_vecm.fit()
print(results_vecm.summary())


Det. terms outside the coint. relation & lagged endog. parameters for equation crude_brent
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
L1.crude_brent      0.0707      0.039      1.801      0.072      -0.006       0.148
L1.soybeans        -0.0173      0.007     -2.305      0.021      -0.032      -0.003
L1.gold             0.0011      0.006      0.182      0.856      -0.011       0.013
L1.silver          -0.0867      0.155     -0.559      0.576      -0.391       0.218
L1.urea_ee_bulk    -0.0044      0.004     -1.026      0.305      -0.013       0.004
L1.maize           -0.0068      0.016     -0.412      0.681      -0.039       0.026
Det. terms outside the coint. relation & lagged endog. parameters for equation soybeans
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------

C:\Users\dhanv\anaconda3\Jupiter\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [56]:
# Forecast using VAR model
lag_order = results_var.k_ar
forecast_var = results_var.forecast(data_diff.values[-lag_order:], steps=5)
forecast_var_df = pd.DataFrame(forecast_var, index=pd.date_range(start=data_diff.index[-1], periods=5, freq='M'), columns=data_diff.columns)
print("VAR Forecast:\n", forecast_var_df)

VAR Forecast:
                                crude_brent   soybeans       gold    silver  \
1970-01-31 00:00:00.000000773     2.199243 -15.043749  19.318632  0.292945   
1970-02-28 00:00:00.000000773     0.615230  25.168538  45.338482  1.078578   
1970-03-31 00:00:00.000000773     3.558246   8.062995  22.491099  1.815577   
1970-04-30 00:00:00.000000773     0.276004  14.819364  25.249291  1.977115   
1970-05-31 00:00:00.000000773    -0.835255  29.997388   0.535326 -0.021538   

                               urea_ee_bulk      maize  
1970-01-31 00:00:00.000000773     -8.124990   8.672008  
1970-02-28 00:00:00.000000773    -26.296339  10.301415  
1970-03-31 00:00:00.000000773    -11.833272  20.869165  
1970-04-30 00:00:00.000000773     44.444075  12.263813  
1970-05-31 00:00:00.000000773     10.647400   3.335629  


C:\Users\dhanv\AppData\Local\Temp\ipykernel_13560\4168714310.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_var_df = pd.DataFrame(forecast_var, index=pd.date_range(start=data_diff.index[-1], periods=5, freq='M'), columns=data_diff.columns)


In [58]:
# Forecast using VECM model
forecast_vecm = results_vecm.predict(steps=5)
forecast_vecm_df = pd.DataFrame(forecast_vecm, index=pd.date_range(start=data_diff.index[-1], periods=5, freq='M'), columns=commodity_data.columns)
print("VECM Forecast:\n", forecast_vecm_df)

VECM Forecast:
                                crude_brent  soybeans       gold    silver  \
1970-01-31 00:00:00.000000773     0.190520 -4.211221 -24.282787 -0.619164   
1970-02-28 00:00:00.000000773     0.003179  0.371492  -6.220449 -0.450121   
1970-03-31 00:00:00.000000773     0.007789  0.259894   0.951758  0.002382   
1970-04-30 00:00:00.000000773    -0.011099 -0.019989   2.305685  0.149529   
1970-05-31 00:00:00.000000773    -0.006130 -0.014182   0.793575  0.055620   

                               urea_ee_bulk     maize  
1970-01-31 00:00:00.000000773      9.587037  1.202406  
1970-02-28 00:00:00.000000773     -1.353035  0.896578  
1970-03-31 00:00:00.000000773     -1.196797 -0.221224  
1970-04-30 00:00:00.000000773     -1.474520 -0.296785  
1970-05-31 00:00:00.000000773     -0.565046 -0.037541  


C:\Users\dhanv\AppData\Local\Temp\ipykernel_13560\69019861.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_vecm_df = pd.DataFrame(forecast_vecm, index=pd.date_range(start=data_diff.index[-1], periods=5, freq='M'), columns=commodity_data.columns)


In [60]:
# Save the results to an Excel file
with pd.ExcelWriter(r"C:\Users\dhanv\Downloads\commodity_forecasts.xlsx") as writer:
    forecast_var_df.to_excel(writer, sheet_name='VAR_Forecast')
    forecast_vecm_df.to_excel(writer, sheet_name='VECM_Forecast')

In [62]:
print("Forecasts saved to commodity_forecasts.xlsx")

Forecasts saved to commodity_forecasts.xlsx
